# CS207 Final Project

## Introduction

**[Prompt] Describe problem the software solves and why it's important to solve that problem**

This package is used for [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation). Two other methods commonly used for numerical differentiation are symbolic differentiation and finite difference methods. The former calculates derivatives to machine precision but is computationally intensive and unreliable. The later is fast and easy but gives poor precision. Automatic differentation is a method that is both computationally efficient and reliable and also gives machine precision.

Applications of numerical differentiation includes optimization, root-finding, and implicit time-integrator, all of which are widely used in science and engineering disciplines.

## Background

**[Prompt] Describe (briefly) the mathematical background and concepts as you see fit. You do not need to give a treatise on automatic differentation or dual numbers. Just give the essential ideas (e.g. the chain rule, the graph structure of calculations, elementary functions, etc). **

#### Chain Rule
A central idea for automatic differentiation is chain rule. Generally, for $h = h\left(y\left(x\right)\right)$ where $y\in\mathbb{R}^{n}$ and $x\in\mathbb{R}^{m}$, its gradient is:

\begin{align}
  \nabla_{x}h = \sum_{i=1}^{n}{\frac{\partial h}{\partial y_{i}}\nabla y_{i}\left(x\right)}.
\end{align}

#### Computational Graph
To represent the operations, a graph structure is sometimes used. For example, for $f\left(x\right) = x - \exp\left(-2\sin^{2}\left(4x\right)\right)$, its graph would be:

![comp-graph](figs/Computational-Graph.png)

#### Overall
Knowing the graph of the computation, and derivatives of elementary functions, we would compute derivative of the entire function using chain rule efficiently.

References: [CS 207 course lecture 9](https://github.com/IACS-CS-207/cs207-F18/blob/master/lectures/L09/L9.ipynb)


## How to Use Our Package - autodiff

**[Prompt] How do you envision that a user will interact with your package? What should they import? How can they instantiate AD objects?**

**Note: This section should be a mix of pseudo code and text. It should not include any actual operations yet.**

The user would import our package, and instantiate variables and operations through our package's interface. The package will automatically compute the derivative for the user.

An example operation:

```Python
import autodiff

x = autodiff.Var()
y = autodiff.Var()
f = 5*x**2+3*y
x.set_value(5)
y.set_value(3)
assert f.der(x) == 50
```

## Software Organization

Discuss how you plan on organizing your software package.

* **What will the directory structure look like?**

    The directory would be like:
  ```
     docs\
          milestone1.ipynb
     README.md
     autodiff\
           autodiff\
                 __init__.py
                 elementary_functions.py
                 module.py
                 tests/
                      __init__.py
           README.md
           setup.py
           LICENSE
           .travis.yml
           setup.cfg
  ```
* **What modules do you plan on including? What is their basic functionality?**

    Modules would include: 
        * elementary_functions which keep the derivative formulas of elementary functions such as cos, sin, exponential and etc.
        * main module that implements the actual automatic differentiation.
    
* **Where will your test suite live? Will you use TravisCI? Coveralls?**

    The test suite will be in the tests directory.
    
    We will use TravisCI for continuous integration and Coveralls for test coverage report.
    

* **How will you distribute your package (e.g. PyPI)?**

    We will distribute our package through PyPI.

## Implementation

Discuss how you plan on implementing the forward mode of automatic differentiation.

* **What are the core data structures?**

    Core data structures include:
        * A list for uniques IDs that identify operations
        
* **What classes will you implement?**

    Classes that we will implement include:
        * Operation - The master class that represents all functions, variables and constants
        * Var - A subclass of Operation, that is used to represent variables
        * Constant - A subclass of Operation, that is used to represent constants
        * Addition/Multiplication/Sin/Cos and etc - Subclasses of Operation that represent elementary functions
        * IDAllocator - A helper class that allocates unique ID to operations/variables
        * Array - A class that represents vector function
* **What method and name attributes will your classes have?**
        
    ```
    Class Operation:
        Attribues: 
            ID
            _value
        Methods:
            __eq__(self, other)
            __add__(self, other)
            __radd__(self, other)
            __mul__(self, other)
            __rmul__(self, other)
            __pow__(self, power, modulo=None)
            __rpow__(self, other)
            evaluate(self)
            der(self, op)
            
    class Var(Operation):
        Methods:
            set_value(self, value)
            
    IDAllocator:
        Attributes:
            ids
        Methods:
            allocate_id(cls)
    
    Array:
        Methods:
            der(self, op)
    ```
* **What external dependencies will you rely on?**
    
    We use numpy for evaluating function values, such as exponential, sine, cosine.
    
    
* **How will you deal with elementary functions like sin and exp?**
    
    We abstract them into custom classes, and implement derivatives for them.


## Additional Comments

There is no need to have an implementation started for this Milestone. You are now in the planning phase. This means that you should feel free to have a project_planning repo in your project organization for scratch work and code. The actual implementation will start after Milestone 1.
